# 1. word2vec을 사용해서 각 나라를 퓨현하는 단어들은 무엇인지 찾기기

In [2]:
!pip install numpy==1.24.4 scipy==1.10.1 gensim==4.3.3 kiwipiepy

In [5]:
import re
import json
import time
import os
import pandas as pd
from kiwipiepy import Kiwi
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [6]:
# KIWI 초기화
kiwi = Kiwi()

# JSON 파일 목록 불러오기
json_files = [f for f in os.listdir('/content') if f.endswith('.json')]

# 텍스트 수집 목록 초기화
texts = []

In [7]:
# 나라별 JSON 파일 읽고 텍스트 정제
for filename in json_files:
    with open(f"/content/{filename}", 'r', encoding='utf-8') as f:
        data = json.load(f)
        for item in data['items']:
            desc = item['description']
            clean_text = re.sub(r"<b>|</b>|&quot;|&lt;|&gt;|(http|https)://[\w\-]+(\.[\w\-]+)+\S*|#", "", desc)
            texts.append(clean_text)

In [8]:
# 형태소 분석 + 단어 리스트 저장
texts_ko = []

for line in texts:
    tokens = kiwi.tokenize(line)
    words = [token.form for token in tokens if token.tag in ['NNG', 'NNP', 'SL']]
    if words:
        texts_ko.append(words)


In [ ]:
# Word2Vec 학습
start = time.time()
model = Word2Vec(
    sentences=texts_ko,
    vector_size=100,
    window=5,
    min_count=1,
    sg=1,
    epochs=5
)
print("Word2Vec 학습 완료:", round(time.time() - start, 2), "초")
model.save("/content/word2vec_final.model")

Word2Vec 학습 완료: 5.92 초


In [ ]:
# 모델 불러오기
model = Word2Vec.load("/content/word2vec_final.model")

# 나라 이름 목록 만들기
country_names = [f.replace('.json', '') for f in json_files]

# 결과 저장
country_keywords = {}

for country in country_names:
    if country in model.wv:
        try:
            top_words = model.wv.most_similar(country, topn=5)
            country_keywords[country] = top_words
        except:
            print(f"{country} → 유사 단어 추출 실패")
    else:
        print(f"{country} 은 모델에 없음")

In [ ]:
# 나라별 유사 단어 5개 결과 출력
for country, simwords in country_keywords.items():
    print(f"[{country}] 유사 단어:")
    for word, score in simwords:
        print(f"  - {word} (유사도: {score:.3f})")

[터키] 유사 단어:
  - 그리스 (유사도: 0.905)
  - 산토리 (유사도: 0.860)
  - 이스탄불 (유사도: 0.842)
  - 지중해 (유사도: 0.842)
  - 신혼여행 (유사도: 0.838)
[뉴질랜드] 유사 단어:
  - 오세아니아 (유사도: 0.928)
  - 이벤트 (유사도: 0.908)
  - 호주 (유사도: 0.904)
  - 관심 (유사도: 0.898)
  - 프로그램 (유사도: 0.897)
[일본] 유사 단어:
  - 도쿄 (유사도: 0.934)
  - 선물 (유사도: 0.923)
  - 오사카 (유사도: 0.916)
  - 후쿠오카 (유사도: 0.903)
  - 리스트 (유사도: 0.882)
[벨기에] 유사 단어:
  - 브뤼셀 (유사도: 0.950)
  - 네덜란드 (유사도: 0.949)
  - 개선문 (유사도: 0.929)
  - 에펠탑 (유사도: 0.929)
  - 암스테르담 (유사도: 0.923)
[오세아니아] 유사 단어:
  - 대륙 (유사도: 0.948)
  - 아시아 (유사도: 0.947)
  - 관심 (유사도: 0.940)
  - 국가 (유사도: 0.939)
  - 남미 (유사도: 0.938)
[상해] 유사 단어:
  - 북경 (유사도: 0.955)
  - 상하이 (유사도: 0.936)
  - 예원 (유사도: 0.934)
  - 마트 (유사도: 0.932)
  - 타이베이 (유사도: 0.930)
[샌프란시스코] 유사 단어:
  - LA (유사도: 0.974)
  - 라스베가스 (유사도: 0.950)
  - 라스베이거스 (유사도: 0.940)
  - 요세미티 (유사도: 0.936)
  - 금 (유사도: 0.910)
[쿠바] 유사 단어:
  - 일 (유사도: 0.931)
  - 멕시코 (유사도: 0.931)
  - 동안 (유사도: 0.926)
  - 그때 (유사도: 0.924)
  - 꿈 (유사도: 0.921)
[베니스] 유사 단어:
  - 베네치아 (유사도: 0.984)
  - 로마 (유사도: 0.958)
  -

# **2. Doc2Vec으로 각 나라와 유사한 국가를 찾기**

In [11]:
from gensim.models.doc2vec import TaggedDocument

In [9]:
# 나라별 문서 만들기 (나라 하나 = 하나의 문서)
documents = []

In [12]:
json_files = [f for f in os.listdir('/content') if f.endswith('.json')]

for filename in json_files:
    country = filename.replace(".json", "")
    with open(f"/content/{filename}", 'r', encoding='utf-8') as f:
        data = json.load(f)
        merged_text = ""
        for item in data['items']:
            desc = re.sub(r"<b>|</b>|&quot;|&lt;|&gt;|(http|https)://[\w\-]+(\.[\w\-]+)+\S*|#", "", item['description'])
            merged_text += desc + " "  # 한 나라의 전체 description 합치기

        tokens = kiwi.tokenize(merged_text)
        words = [token.form for token in tokens if token.tag in ['NNG', 'NNP', 'SL']]
        documents.append(TaggedDocument(words=words, tags=[country]))  # 나라 이름이 tag가 됨

In [13]:
model = Doc2Vec(
    documents,
    vector_size=100,
    window=5,
    min_count=1,
    workers=2,
    epochs=10
)

# 저장
model.save("/content/doc2vec_countries.model")

In [16]:
# 모델 불러오기
model = Doc2Vec.load("/content/doc2vec_countries.model")

target_country = "일본"
similar = model.dv.most_similar(target_country, topn=1)  # topn=1로 수정

print(f"{target_country}과 가장 유사한 지역은:")
for other_country, score in similar:
    print(f" - {other_country} (유사도: {score:.3f})")


일본과 가장 유사한 지역은:
 - 도쿄 (유사도: 0.987)


In [17]:
# 전체 나라에 대해 유사한 지역 출력
all_countries = model.dv.index_to_key

for country in all_countries:
    similar = model.dv.most_similar(country, topn=1)
    most_similar_country, score = similar[0]
    print(f"{country} → {most_similar_country} (유사도: {score:.3f})")


이스라엘 → 예루살렘 (유사도: 0.991)
중남미 → 페루 (유사도: 0.983)
그랜드케년 → 옐로스톤 (유사도: 0.993)
베니스 → 피렌체 (유사도: 0.986)
싱가포르 → 홍콩 (유사도: 0.982)
터키 → 아테네 (유사도: 0.931)
산토리니 → 그리스 (유사도: 0.974)
캐나다 → 아프리카 (유사도: 0.967)
상해 → 중국 (유사도: 0.977)
뉴욕 → 아이슬란드 (유사도: 0.975)
폴란드 → 오스트리아 (유사도: 0.971)
시드니 → 호주 (유사도: 0.986)
미국 → 샌프란시스코 (유사도: 0.980)
이집트 → 멕시코 (유사도: 0.970)
제주 → 홍콩 (유사도: 0.964)
도쿄 → 후쿠오카 (유사도: 0.990)
네팔 → 히말라야 (유사도: 0.986)
독일 → 베를린 (유사도: 0.972)
포루투갈 → 스페인 (유사도: 0.984)
피렌체 → 로마 (유사도: 0.987)
브라질 → 중남미 (유사도: 0.983)
파리 → 프랑스 (유사도: 0.995)
모로코 → 노르웨이 (유사도: 0.973)
프라하 → 체코 (유사도: 0.994)
로마 → 바티칸 (유사도: 0.989)
타이페이 → 대만 (유사도: 0.992)
홍콩 → 싱가포르 (유사도: 0.982)
아프리카 → 뉴질랜드 (유사도: 0.985)
하노이 → 다낭 (유사도: 0.985)
말레이지아 → 동남아시아 (유사도: 0.979)
러시아 → 인도 (유사도: 0.986)
쿠바 → 멕시코 (유사도: 0.992)
네덜란드 → 벨기에 (유사도: 0.981)
중국 → 홍콩 (유사도: 0.982)
동남아시아 → 말레이지아 (유사도: 0.979)
라스베이거스 → LA (유사도: 0.993)
영국 → 런던 (유사도: 0.993)
요세미티 → LA (유사도: 0.991)
인도네시아 → 동남아시아 (유사도: 0.971)
서유럽 → 피렌체 (유사도: 0.977)
방콕 → 태국 (유사도: 0.988)
예루살렘 → 이스라엘 (유사도: 0.991)
동유럽 → 프라하 (유사도: 0.985)